## Elevation Pressure

[Elevation Pressure](https://www.apmonitor.com/dde/index.php/Main/ElevationPressure) in the [Data-Driven Engineering](http://apmonitor.com/dde) online course.

<img align=left width=500px src='https://apmonitor.com/dde/uploads/Main/elevation_pressure.png'>

Autonomous navigation requires a variety of sensors to create an estimate of absolute or relative position. Each sensor has relative strengths and weaknesses and sensor fusion ties the information together to create a current best estimate of position. Pressure is commonly used in cell phones, watches, drones, and other electronics to determine change in elevation. The barometric pressure varies slowly with weather patterns so absolute altitude needs a reference such as GPS or a barometric pressure reading at a known altitude. Start by importing libraries needed for the analysis.

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The BMP280 sensor is a Bosch piezo-resistive absolute pressure sensor with low power consumption and long-term measurement stability. It is 0.95mm x 2mm x 2.5mm in size and also includes temperature. The BME280 adds humidity as well.

This case study is an analysis of the BMP280 sensor to determine relative and absolute elevation resolution from the BMP280 sensor.

<img align=left width=200px src='https://apmonitor.com/dde/uploads/Main/pressure_BMP280.png'>

**Activity: Calculate Elevation from Pressure**

Calculate the elevation from the BMP280 pressure and compare to GPS elevation. Data from the BMP280 sensor is in the ''Pressure (Pa)'' column and it is recorded with GPS and vehicle information.

### Import Data File

The vehicle drives from the bottom of the hill to the top 3 times. Import the data for the pressure and data collected from the vehicle.

In [ ]:
file = 'http://apmonitor.com/dde/uploads/Main/elevation_pressure.txt'
data = pd.read_csv(file)
data.head()

### View Data

Create a plot of the Pressure (Pa) from the BMP280 sensor and Altitude (ft) from the GPS measurement.

In [ ]:
data[['Pressure (Pa)','Altitude (ft)']].plot(figsize=(10,5),subplots=True)

### Calculate Height from Pressure

Calculate the height (ft) based on pressure (Pa). There are 3.28084 ft per meter. Rearranging the pressure to elevation equation gives the height equation

$h = \left(\frac{T_0}{L}\right)\left(1-\left(\frac{P}{P_0}\right)^{\frac{R_g L}{g\,M}}\right)$

with

* height ($h$) in meters
* sea level standard atmospheric pressure ($P_0=101325$ Pa)
* temperature lapse rate ($L=0.00976$ K/m)
* sea level standard temperature ($T_0=288.16$ K)
* gravitational acceleration ($g=9.80665$ $\frac{m}{s^2}$)
* molar mass of dry air ($M=0.02896968$ kg/mol)
* universal gas constant ($R_g=8.31446$ $\frac{J}{mol\,K}$)

In [ ]:
T0 = 288.16; L=0.00976; P0=101325; Rg=8.31446
g  = 9.80665; M=0.02896968
data['h'] = (T0/L)*(1-(data['Pressure (Pa)']/P0)**(Rg*L/(g*M))) * 3.28084

### Bias Correction

Determine the one-time bias that corrects for the change in barometric pressure from the first data point. $h_{GPS,0}$ is the initial GPS altitude and $h_{u,0}$ is the initial calculated height from the pressure.

$bias = h_{GPS,0}-h_{u,0}$

In [ ]:
bias = data['Altitude (ft)'].iloc[0]-data['h'].iloc[0]

### Add Bias Correction

Add the ''bias'' to every calculated height unbiased height $(h_{u})$ to calculate the correct height $(h_b)$.

$h_{b} = h_{u}+bias$

In [ ]:
data['Height (Pressure)'] = data['h'] + bias

### View Results

Compare the biased height $(h_b)$ with the measured GPS height. View the results on a plot.

In [ ]:
data[['Height (Pressure)','Altitude (ft)']].plot()

### Simulate Barometric Pressure Change

The barometric pressure varies slowly with weather patterns (+/- 1700 Pa). Show how a change of +1000 Pa affects the height calculation.

In [ ]:
data['Height (+1000 Pa)'] = (T0/L)*(1-((data['Pressure (Pa)']+1000)/P0)**(Rg*L/(g*M))) * 3.28084 + bias
data[['Height (+1000 Pa)','Altitude (ft)']].plot()

### Display GPS Route

In [ ]:
#pip install plotly

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(data, lat="Latitude (deg)", lon="Longitude (deg)", \
                        color="Vehicle speed (MPH)", size="Altitude (ft)", \
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=5, zoom=14)
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()